In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd

In [2]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance/60)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance/60))

In [3]:
def create_data_model():
    matrix = pd.read_csv('distance.csv')
    m_list = []
    
    for i in range(len(matrix)):
        temp = []
        for j in range(len(matrix)):
            temp.append(matrix.iloc[i,j])
        m_list.append(temp)
    print(len(m_list))
    data = {}
    data['distance_matrix'] = m_list
    data['num_vehicles'] = 15
    data['depot'] = 50
    return data

In [4]:
# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

51


In [5]:
def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node] 

In [6]:
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [7]:
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    100000,  # no slack
    30000000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [8]:
def print_solution_only_num():
    output = []
    for vehicle_id in range(data['num_vehicles']):
        Output = []
        index = routing.Start(vehicle_id)
        while not routing.IsEnd(index):
            Output.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        Output.append(manager.IndexToNode(index))  
        output.append(Output)
    return output

In [9]:
solution = routing.SolveWithParameters(search_parameters)

In [10]:
if solution:
    print_solution(data, manager, routing, solution)
    solution = print_solution_only_num()
else:
    print('No solution found !')

Objective: 175833
Route for vehicle 0:
 50 -> 50
Distance of the route: 0.0m

Route for vehicle 1:
 50 ->  1 ->  41 ->  23 ->  39 -> 50
Distance of the route: 19.5m

Route for vehicle 2:
 50 ->  15 ->  20 ->  22 ->  6 ->  43 ->  35 -> 50
Distance of the route: 17.766666666666666m

Route for vehicle 3:
 50 ->  18 ->  40 ->  4 -> 50
Distance of the route: 23.5m

Route for vehicle 4:
 50 ->  19 -> 50
Distance of the route: 20.9m

Route for vehicle 5:
 50 ->  45 ->  49 ->  26 ->  38 ->  0 ->  27 ->  37 ->  8 -> 50
Distance of the route: 21.383333333333333m

Route for vehicle 6:
 50 ->  31 ->  14 ->  7 -> 50
Distance of the route: 20.666666666666668m

Route for vehicle 7:
 50 ->  46 ->  17 -> 50
Distance of the route: 26.15m

Route for vehicle 8:
 50 ->  32 ->  13 ->  9 -> 50
Distance of the route: 26.333333333333332m

Route for vehicle 9:
 50 ->  5 -> 50
Distance of the route: 17.433333333333334m

Route for vehicle 10:
 50 -> 50
Distance of the route: 0.0m

Route for vehicle 11:
 50 ->  16

In [11]:
import folium

In [12]:
cluster = pd.read_csv('clustered.csv')
g_cluster = cluster[['lat','lng']]
g_cluster = g_cluster.append({'lng':126.847276,'lat':35.229482},ignore_index=True)
g_cluster

/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_3264/3764917458.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  g_cluster = g_cluster.append({'lng':126.847276,'lat':35.229482},ignore_index=True)


,lat,lng
0,35.218975,126.845136
1,35.209160,126.868205
2,35.209191,126.839777
3,35.197043,126.839997
4,35.210472,126.877059
5,35.180530,126.816932
6,35.206737,126.860138
7,35.211772,126.833553
8,35.214210,126.843679
9,35.215043,126.851159


In [13]:
cluster

,lat,lng,cluster,clust_s
0,35.218975,126.845136,29.960000,26.186667
1,35.209160,126.868205,14.187500,22.000000
2,35.209191,126.839777,26.187500,15.093750
3,35.197043,126.839997,9.000000,20.000000
4,35.210472,126.877059,12.000000,22.000000
5,35.180530,126.816932,25.000000,43.000000
6,35.206737,126.860138,4.000000,2.000000
7,35.211772,126.833553,13.000000,17.000000
8,35.214210,126.843679,39.982456,16.947368
9,35.215043,126.851159,3.087379,10.000000


In [14]:
mean_x = g_cluster['lng'].mean()
mean_y = g_cluster['lat'].mean()

In [2]:
m = folium.Map( location = [mean_y,mean_x],zoom_start=14)
for i in range(len(cluster)):
    folium.Marker(
        location = [g_cluster.iloc[i]['lng'],g_cluster.iloc[i]['lat']]
    ).add_to(m)
    
m

NameError: name 'folium' is not defined

In [16]:
color = ['green','red','blue','yellow','pink','pupple','black','orange','blue']
num = 0
for i in solution:
    
    for j in range(1,len(i)-1):
        print(i,j)
        folium.PolyLine(locations=[[g_cluster.iloc[i[j]]['lat'],g_cluster.iloc[i[j]]['lng']], [g_cluster.iloc[i[j+1]]['lat'],g_cluster.iloc[i[j+1]]['lng']]]
                        ,tooltip='Polyline'
#                         ,color = color[num]
                       ).add_to(m)
    num+=1

[50, 1, 41, 23, 39, 50] 1
[50, 1, 41, 23, 39, 50] 2
[50, 1, 41, 23, 39, 50] 3
[50, 1, 41, 23, 39, 50] 4
[50, 15, 20, 22, 6, 43, 35, 50] 1
[50, 15, 20, 22, 6, 43, 35, 50] 2
[50, 15, 20, 22, 6, 43, 35, 50] 3
[50, 15, 20, 22, 6, 43, 35, 50] 4
[50, 15, 20, 22, 6, 43, 35, 50] 5
[50, 15, 20, 22, 6, 43, 35, 50] 6
[50, 18, 40, 4, 50] 1
[50, 18, 40, 4, 50] 2
[50, 18, 40, 4, 50] 3
[50, 19, 50] 1
[50, 45, 49, 26, 38, 0, 27, 37, 8, 50] 1
[50, 45, 49, 26, 38, 0, 27, 37, 8, 50] 2
[50, 45, 49, 26, 38, 0, 27, 37, 8, 50] 3
[50, 45, 49, 26, 38, 0, 27, 37, 8, 50] 4
[50, 45, 49, 26, 38, 0, 27, 37, 8, 50] 5
[50, 45, 49, 26, 38, 0, 27, 37, 8, 50] 6
[50, 45, 49, 26, 38, 0, 27, 37, 8, 50] 7
[50, 45, 49, 26, 38, 0, 27, 37, 8, 50] 8
[50, 31, 14, 7, 50] 1
[50, 31, 14, 7, 50] 2
[50, 31, 14, 7, 50] 3
[50, 46, 17, 50] 1
[50, 46, 17, 50] 2
[50, 32, 13, 9, 50] 1
[50, 32, 13, 9, 50] 2
[50, 32, 13, 9, 50] 3
[50, 5, 50] 1
[50, 16, 25, 50] 1
[50, 16, 25, 50] 2
[50, 10, 24, 42, 3, 21, 50] 1
[50, 10, 24, 42, 3, 21, 50] 2
[

In [17]:
solution = pd.DataFrame(solution)
solution

,0,1,2,3,4,5,6,7,8,9,10
0,50,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50,1,41.0,23.0,39.0,50.0,NaN,NaN,NaN,NaN,NaN
2,50,15,20.0,22.0,6.0,43.0,35.0,50.0,NaN,NaN,NaN
3,50,18,40.0,4.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
4,50,19,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,50,45,49.0,26.0,38.0,0.0,27.0,37.0,8.0,50.0,NaN
6,50,31,14.0,7.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
7,50,46,17.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,50,32,13.0,9.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
9,50,5,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
solution.to_csv("solution_cluster_info.csv",mode='w')